In [ ]:
from sqlalchemy import create_engine, select, __version__
from sqlalchemy.orm import sessionmaker
import os, sys

print(__version__)

sys.path.append(os.path.join(os.path.abspath(""), "models"))
sys.path.append(os.path.join(os.path.abspath(""), "search_tool"))

from models import *
from search_tool import *

fpath = r"\\btssce\csbpim1\DEIL\Data\Prod\Projects\DEIL_INFC_Green\4-Collection\GOAT PIPELINE TEST\lode_master_database.sqlite"

print(fpath)

engine = create_engine(f"sqlite:///{fpath}", echo=False)

#META_DATA = MetaData()

#META_DATA.reflect(bind=engine)

#print(META_DATA.tables.keys())

#StandardGeographicClassification = Table('standard_geographic_classification', META_DATA, autoload=True)

#print(StandardGeographicClassification.columns.keys())

#query = select(StandardGeographicClassification).where(StandardGeographicClassification.sgc_name.like("%montr%al%"))

sesh = sessionmaker(bind=engine)()

#result = sesh.scalars(query)

#for i in result:
    #print(i) #.columns.keys())
#    print(i.dguid, i.sgc_name, i.sgc_type_uid, i.sgc_level.sgc_type_name, i.sgc_subtype_uid, i.sgc_subtype.sgc_subtype_name)    




In [ ]:
from ipywidgets import interact_manual
import ipywidgets as widgets

wid_provider = widgets.Text(
    value='',
    placeholder='Type something',
    description='Search:',
    disabled=False   
)

@interact_manual(x=wid_provider)
def blort(x, search_providers=True, search_sgc=False, search_licences=False):
    if search_providers:
        print("\nPROVIDERS ========\n")
        fun_search_provider(sesh, x)
    if search_sgc:
        print("\nSTANDARD GEOGRAPHIC CLASSIFICATION ========\n")
        fun_search_area(sesh, x)
    if search_licences:
        print("\nLICENCES ========\n")
        fun_search_licence(sesh, x)

# Provider Helper Tool

In [ ]:
import pandas as pd

import json

from IPython.display import clear_output

def write_clean_json(df, output_file):
    """Write a dataframe to a json file with clean formatting"""

    # drop picket fence backslashes
    # ref: https://stackoverflow.com/a/64826042 
    with open(output_file, "w", encoding="utf8") as o_f:
        json_str = df.to_json(orient="records") 
        o_f.write(json.dumps(json.loads(json_str), ensure_ascii=False, indent=4))

df = pd.read_json(r"C:\Users\barisma\Documents\deil_infc_green\Phase 1\Industrial Zones\sources.json", orient="records")

for idx, r  in df.iterrows():
    if r["provider_uid"] == "":
        res = [i for i in f(r["area"], print_results=False)]

        print(r, flush=True)
        
        if len(res) == 1:
            acc = ""
            while acc.lower() not in ["y", "n", "q"]:
                acc = input("FOUND: %s  ---  Do you accept? [y/n/q]" % res[0].provider_name)
                #print(acc)
            if acc.lower() == "y":
                df.loc[df.index == idx, "provider_uid"] = res[0].uid
            elif acc.lower() == "q":
                break    
            else:
                pass

        clear_output(wait=True)

write_clean_json(df, r"C:\Users\barisma\Documents\deil_infc_green\Phase 1\Industrial Zones\sources_fixed.json")

# Licence Helper Tool

In [ ]:
import pandas as pd

import json

from IPython.display import clear_output

def write_clean_json(df, output_file):
    """Write a dataframe to a json file with clean formatting"""

    # drop picket fence backslashes
    # ref: https://stackoverflow.com/a/64826042 
    with open(output_file, "w", encoding="utf8") as o_f:
        json_str = df.to_json(orient="records") 
        o_f.write(json.dumps(json.loads(json_str), ensure_ascii=False, indent=4))

df = pd.read_json(r"C:\Users\barisma\Documents\goat\data_sources_transformer\output\air_pollution\licenses.json", orient="records")

for idx, r  in df.iterrows():
    res = [i for i in h(r["licence_url"], print_results=False)]

    print(r, flush=True)
    print(len(res))
    
    if len(res) == 1:
        acc = ""
        while acc.lower() not in ["y", "n", "q"]:
            acc = input("FOUND: %s  ---  Do you accept? [y/n/q]" % res[0].licence_url)
            #print(acc)
        if acc.lower() == "y":
            df.loc[df.index == idx, "licence_uid"] = res[0].uid
        elif acc.lower() == "q":
            break    
        else:
            pass

    #clear_output(wait=True)

#write_clean_json(df, r"C:\Users\barisma\Documents\deil_infc_green\Phase 1\Industrial Zones\sources_fixed.json")